In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab5-controls.ipynb")

# EE 120 Lab 5: Feedback and Control

**Signals and Systems** at UC Berkeley

Acknowledgements:
* **Spring 2019** (v1.0): Jonathan Lee, Akash Velu, Babak Ayazifar
* **Fall 2019** (v1.1): Jonathan Lee, Murat Arcak
* **Spring 2020** (v2.0): Jonathan Lee, Babak Ayazifar
* **Spring 2022** (v3.0): Gaurav Datta
* **Spring 2023** (v3.1): Edward Lee

# Introduction

Many interesting physical systems use feedback to regulate or stabilize themselves:

* Mechatronics: vehicle autopilots, robotic joints
* Communication networks: TCP congestion control, [signal synchronization](https://en.wikipedia.org/wiki/Phase-locked_loop)
* Biological networks: predator-prey models, gene regulation

Feedback is particularly useful for **closed-loop control**. In closed loop control, we use sensors to collect information about the system state and use these observations to determine what input we should send so the system in some desired way.
Closed-loop control has a number of advantages over open-loop control, which does not use feedback:
* Controllers in closed-loop systems are easier to design. We can treat the system we are trying to control (called a plant) as a black box.
* Designed well, closed-loop systems tend to be more robust to modeling imperfections, disturbances, and noise.
* Feedback loops are often automated by computers, so a closed-loop system&mdash;such as the cruise control of a car&mdash;can compensate for error at a much greater frequency than a human operator can.

## Background: Laplace Transform

The bilateral Laplace transform of a continuous-time signal $x(t)$ is:
$$X(s) \triangleq \int_{-\infty}^\infty x(t) e^{-st} \mathrm{d}t$$
We can view the Laplace transform as a generalization from the continuous-time Fourier transform, since the Laplace transform can represent a broader class of signals (in particular, signals with exponential envelopes). This is because while the Fourier transform represents a signal in terms of $e^{i\omega t}$, the Laplace transform represents a signal in terms of $e^{st}$ where $s\in\mathbb{C}$.
For the special case $X(s)|_{s=i\omega}$, we recover the CTFT of $x(t)$.

Like the z-transform, its discrete-time analog, every Laplace transform is associated with a region of convergence (ROC)&mdash;a set of $s \in \mathbb{C}$ where the Laplace integral converges.
For rational Laplace transforms, the numerator and denominator polynomials can be factored into their roots, zeros and poles.
ROCs are bounded by vertical lines on which poles sit, so ROCs take the form of a right-sided half-plane, left-sided half-plane, or vertical strip.

For a CT-LTI system, the Laplace transform of its impulse response is called the transfer function of the system. This is often more useful than just the frequency response because it tells us how the system responds to signals that are complex exponentials: For an input $x(t) = e^{st}$, where $s=\sigma + i \omega$ is an arbitrary complex number, the corresponding output is $y(t) = H(s) e^{st}$ where $H$ is the Laplace transform of the system's impulse response $h$ (assuming the Laplace transform converges at $s$) [Prove it!].

Once critical property of the Laplace transform is the differentiation in time property.
$$\frac{dx}{dt} \Leftrightarrow sX(s) -x(0^{-})$$

If we apply this property to a causal Single-Input-Single-Output (SISO) CT-LTI system $H$ that is governed by an LCCDE,
$$
\sum_{m=0}^M a_m \frac{\mathrm{d}^m y(t)}{\mathrm{d}t^m} = \sum_{n=0}^N b_n \frac{\mathrm{d}^n x(t)}{\mathrm{d}t^n},
$$
the transfer function is a rational functions of $s$,
$$
H(s) = \frac{\sum_{n=0}^N b_n s^n}{\sum_{m=0}^M a_m s^m}
$$

The values of $s$ for which the transfer function $H(s)$ is undefined are called the poles of the system. For systems governed by LCCDEs, there are the roots of the denominator of the transfer function.

## Background: State Space

During our study of CT-LTI systems, we saw how systems described by differential equations can be written compactly in a standard state-space form:
$$
\begin{aligned}
\frac{\mathrm{d} \mathbf{x}(t)}{\mathrm{d}t} &= \mathbf{A} \mathbf{x}(t) + \mathbf{B} u(t), \\
y(t) &= \mathbf{C} \mathbf{x}(t) + D u(t),\\
\mathbf{x}(0) &= \mathbf{x}_0,
\end{aligned}
$$
where:
* $u(\cdot)$ is a scalar input signal (**not** necessarily the unit step),
* $y(\cdot)$ is a scalar output signal,
* $\mathbf{x}(\cdot): \mathbb{R} \to \mathbb{R}^n$ are $n$ intermediate states (also known as the system **trajectory**),
* $\mathbf{x}_0 \in \mathbb{R}^n$ are the initial conditions, and
* $\mathbf{A}, \mathbf{B}, \mathbf{C}, D$ are appropriately sized matrices of constants.

Applying the Laplace transform to each state variable, you can compute the transfer function of the overall system $u(t) \to y(t)$ after assuming a zero initial condition $\mathbf{x}_0 = \mathbf{0}$.
$$\begin{aligned}
s\mathbf{X}(s) &= \mathbf{A} \mathbf{X}(s) + \mathbf{B} U(s) \\
Y(s) &= \mathbf{C} \mathbf{X}(s) + D U(s) \\
H(s) = \frac{Y(s)}{U(s)} &= \mathbf{C} (s\mathbf{I} - A)^{-1} \mathbf{B} + D
\end{aligned}$$
The beauty of the Laplace transform is that our differential equation has been turned into an algebraic one of polynomials in $s$, which greatly simplifies analysis and design.
For example, the poles of $H(s) = Y(s)/U(s)$ are a subset of the eigenvalues of $\mathbf{A}$:
$$(s\mathbf{I} - \mathbf{A})^{-1} = \frac{\mathrm{adj}{(s\mathbf{I} - \mathbf{A})}}{\det{(s\mathbf{I} - \mathbf{A})}}$$
(The adjugate of a matrix $A$, denoted $\mathrm{adj}(A)$ is computed from the cofactors of $A$, which are used in Laplace's formula for computing the determinant. See [Wikipedia](https://en.wikipedia.org/wiki/Adjugate_matrix) for more details.)

It is easy to check whether a causal system $u(t) \to y(t)$ is BIBO-stable by checking whether the eigenvalues that are poles lie in the open left-half plane. Equivalently, we want the ROC of the Laplace Transform to include the imaginary axis.

## Background: Satellite Stabilization

<table style="margin: 1em 0">
    <tr>
        <td><img src="diagrams/orbit.png" /></td>
        <td><img src="diagrams/flywheels.png" /></td>
    </tr>
</table>
<center>Figure 1. Left: Relative orientation of Earth and satellite frames. Right: Flywheel schematic.</center>

In this lab, we'll use linear control to stabilize a satellite, as described in [1].
For many satellites, it is critical to maintain a certain orientation with respect to the Earth.
For example, an imaging satellite might need to keep its camera pointed towards the Earth's surface.
Unfortunately, disturbances from the environment, such as solar radiation or the Earth's magnetic field, can exert torques on the satellite that cause undesired rotation.
For this reason, some satellites are equipped with flywheels, like those shown in Figure 1 (right), to absorb or release rotational energy and induce a compensating torque.

Using the notation in Figure 1 (left), we would like to rotate the satellite about the $Y$-axis such that $\theta = 0$ and the $Z$ and $Z_0$ axes are parallel (the $Z$ and $Z_0$ axes lie in the plane of the orbit).
For simplicity, we'll ignore the orientation of the $X$ and $Y$ axes.

In [2]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import integrate, signal

import warnings
# NOTE: do not remove this line; otter-grader breaks without it.
#
# Some phase plane code throws an extraneous "badly conditioned filter 
# coefficients" warning that should be ignored.
warnings.filterwarnings("ignore", message="Badly conditioned filter coefficients")

matplotlib.rc('font', size=20)
if 'ggplot' in plt.style.available:
    plt.style.use('ggplot')

%matplotlib inline

## Q1: Simulating the Open-Loop Vehicle Dynamics
We'll start by looking at the open-loop vehicle dynamics. This is what happens if we apply our input without any feedback.

![Open Loop System](diagrams/open-loop.png)

From Newton's second law, the differential equation which governs our open-loop system is

$$I_y \ddot{\theta}(t) = u(t),$$

where $I_y > 0$ is the satellite's moment of inertia and $u(t)$ is the net torque (**not** necesarily the unit step), both about the $Y$-axis.
Note that this system is a double integrator with a gain.

This system can be expressed in standard state-space form

$$\frac{\mathrm{d}\mathbf{x}(t)}{\mathrm{d}t} = \mathbf{A}_\mathsf{sat} \mathbf{x}(t) + \mathbf{B}_\mathsf{sat} u(t),\ \mathbf{x}(t_0) = \mathbf{x}_0$$

using $x_1 = \theta, x_2 = \dot{\theta}$ as the state variables.

### Q1(a): Building the Model
Fill in the matrices in the following code block to complete the open-loop model.

In [ ]:
I_y = 1.28e4  # Representative value in kg m^2
A_sat = ...
B_sat = ...

### Q1(b): Simulating the System
Next, we'll write a method that will allow us to solve a general standard state-space differential equation

$$\dot{\mathbf{x}} = \mathbf{Ax}(t) + \mathbf{Bu}(t).$$

Use [`scipy.integrate.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) with the `t_eval` argument to implement the solver.
The default numerical integration method works well enough, so there is no need to use a different method.

When creating your $\dot{\mathbf{x}}$ function, use `np.dot` instead of `@` to multiply matrices.

In [ ]:
def solve_lti(A, B, u, t, x0):
    """
    Solve an LTI state-space equation.
    
    Arguments:
        A (numpy.ndarray): State transition matrix with shape (n, n)
        B (numpy.ndarray): Input-to-state map with shape (n,)
        u: Two-argument function that accepts the current time and
            state of shape (n,), and returns the control input.
        t (numpy.ndarray): Times the state should be evaluated at
            with shape (T,).
        x0 (numpy.ndarray): Initial state with shape (n,).
    
    Returns:
        The state variables over time with shape (n, T).
    """
    ...

In [ ]:
grader.check("q1b-code")

Now that we have our open-loop model, lets simulate what happens to it. Ideally, we would like it to remain at the origin $(\theta, \dot{\theta}) = (0, 0)$.

The next two cells simulate the satellite with a constant disturbance due to constant radiation pressure.
1. The first plot depicts $\theta$ versus $\dot{\theta}$ and is called a **phase portrait**. Each curve represents one simulation from a different initial condition. Arrows are placed a uniform amount of time apart so you can estimate the rate of change in each state. For planar systems (systems with exactly two states), phase portraits are often more informative than plotting each state over time individually because you can see how the two states interact.
2. The second plot is simply $\theta$ over time for all trajectories.

In [ ]:
def simulate_satellite(A, B, u, t, init_states, width=10, height=15,
                       step=50, titles=None, **quiver_options):
    trajectories = np.empty((init_states.shape[0], 2, t.size))
    figure, (phase_ax, theta_ax) = plt.subplots(2, 1, figsize=(width, height),
                                                gridspec_kw={'height_ratios': [1.5, 1]})
    
    # Label axes
    phase_ax.set_xlabel('Anglular Displacement (rad)')
    phase_ax.set_ylabel('Angular Velocity (rad/s)')
    theta_ax.set_xlabel('Hours (h)')
    theta_ax.set_ylabel('Anglular Displacement (rad)')
    
    # Compute the trajectory from each initial state
    for i, x0 in enumerate(init_states):
        trajectories[i] = x = solve_lti(A, B, u, t, x0)
        phase_ax.plot(x[0], x[1], color='tab:blue', alpha=0.3)
        theta_ax.plot(t/3600, x[0], color='tab:blue', alpha=0.3)
    
    # Draw uniformly spaced arrows along the trajectory
    base = trajectories[:, :, ::step]
    direction = trajectories[:, :, 1::step] - base
    phase_ax.quiver(base[:, 0], base[:, 1], direction[:, 0], direction[:, 1],
                    color='tab:blue', angles='xy', **quiver_options)
    
    # Draw initial states
    phase_ax.scatter(init_states[:, 0], init_states[:, 1], marker='x', label='Initial states')
    phase_ax.legend()
    
    return trajectories, phase_ax, theta_ax

In [ ]:
# Constant torque in Newton-meters due to radiation pressure on a reflective surface.
w = lambda t, x: 4.3e-6       

# Time scale and initial states
t = np.linspace(0, 7*24*60*60, 2000)  # In seconds
theta_init = np.linspace(-0.5*np.pi, 0.5*np.pi, 5)
theta_dot_init = np.linspace(-1e-4, 1e-4, 7)
init_states = np.stack(np.meshgrid(theta_init, theta_dot_init)).reshape(2, -1).T

plt.figure(figsize=(10, 6))
_, phase_ax, theta_ax = simulate_satellite(A_sat, B_sat, w, t, init_states)
phase_ax.set_xlim(-6*np.pi, 6*np.pi)
phase_ax.set_ylim(-1.1e-4, 1.5e-4)

phase_ax.set_title("Phase portrait of open-loop under constant disturbance.")
theta_ax.set_title("Angular Displacement of open-loop under constant disturbance.")

<!-- BEGIN QUESTION -->

### **Q1(c):** Open Loop Analysis
**Q**: Recall the poles of the transfer function are a subset of the eigenvalues of $\mathbf{A}_{\mathsf{sat}}$. What are the eigenvalues of $\mathbf{A}_{\mathsf{sat}}$? What is the transfer function $H(s) = \frac{\Theta(s)}{U(s)}$? Comment on the BIBO stability of the open-loop system and how the phase portrait and the pole placement individually explain the stability.

Remember, ideally, we would like trajectories to converge to the origin $(\theta, \dot{\theta}) = (0, 0)$, meaning the desired and actual axes, $Z_0$ and $Z$, are aligned, and the alignment is not changing over time.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Q2: Closing the Loop

Although we simulated the open-loop system with a simple constant disturbance, in general, the disturbance is difficult to model and estimate because of the complex dynamics involved.
We could try to correct the disturbance by applying a constant torque to cancel what we think the disturbance torque is to keep the observed angle $\theta$ where we want it. This is an example of open-loop control, where the control input we apply is independent of the variable we are trying to control. However, if our assumption of a constant disturbance is inaccurate, or we are unable to exactly cancel it with our applied torque, there will be a net torque that will cause the angle to drift from the value we want.

Since it's not practical to cancel the disturbance with subtraction in the open-loop scheme, we try feedback (or closed-loop) control, where the control input is dependent on the variable we are trying to control:

![Closed Loop System](diagrams/closed-loop.png)

The controller takes the error signal $e(t)$ and outputs a corresponding angular velocity signal $\omega(t)$ to spin the flywheel at.

Note that the flywheel has its own dynamics: it has its own moment of inertia $I_f > 0$ and angular velocity $\omega(t)$.

The transfer function of $\theta(t)$ w.r.t. the disturbance $w(t)$ is

$$\Theta(s) = \frac{1}{I_y s^2} \left[W(s) - I_f s C(s) \Theta(s)\right] \implies H_{w\to\theta}(s) \triangleq \frac{\Theta(s)}{W(s)} = \frac{1}{I_y s^2 + I_f s C(s)}$$

Since the transfer function of our controller $C(s)$ appears the in the denominator of the resulting transfer function, that means we can choose it to carefully place the poles of the system wherever we choose!

There are many choices for the controller $C(s)$.
The scheme we'll use is called proportional-integral-derivative (PID) control, which uses a linear combination of the error signal, integrated error, and error derivative (current, past, and future error) to determine the velocity $\omega(t)$:
$$\omega(t) = k_p e(t) + k_i \int_0^t e(t^\prime) \mathrm{d}t^\prime + k_d \frac{\mathrm{d}e(t)}{dt},$$
where $k_p, k_i, k_d \geq 0$ are coefficients (in the appropriate units) that determine the relative importance of each term.

The Laplace transforms of PID controllers take the form

$$C(s) = k_p + \frac{k_i}{s} + k_ds.$$

This comes from applying the differentiation in time property in conjuction with the integration in time property,

$$\int_0^{t}x(t)dt \Leftrightarrow \frac{1}{s}X(s)$$

This means if we apply a PID controller to the system, our transfer function will become

\begin{align*}
    H_{w\to\theta}(s) &= \frac{1}{I_y s^2 + I_f s C(s)}\\
    &= \frac{1}{I_y s^2 + I_f s \left(k_p + \frac{k_i}{s} + k_ds\right)}\\
    &= \frac{1}{(I_y + k_d I_f)s^2 + k_p I_f s + k_iI_f}
\end{align*}

This corresponds to the LCCDE

$$(I_y + k_d I_f)\ddot{\theta} + k_p I_f \dot{\theta} + k_i I_f \theta = w.$$

Proportional (P), proportional-derivative (PD), and proportional-integral (PI) are also common variants where one or more terms is excluded.
PID and its variants are quite popular in practical controller implementations because it is simple to approximate differentiation and integration with finite differencing and accumulators.

### Q2(a): Disturbance Rejection

Now, we want to simulate the closed-loop system with different sets of PID coefficients and see how well it does at rejecting a constant disturbance
1. Choose as states $x_1 = \theta, x_2 = \dot{\theta}$ and write the controlled system in state-space form. Use this work to implement `make_closed_loop_sys`, which defines the A and B matrices for the controlled system.
2. Run the cell to simulate the closed-loop system and plot its trajectories for some $(k_p, k_i, k_d)$ combinations we've given you. Feel free to add your own coefficients to `k` and experiment.

In [ ]:
I_f = 1e-5*I_y  # I_f << I_y

def make_closed_loop_sys(k_p, k_i, k_d):
    """
    Compute the state-space matrices for the closed-loop system
    under PID control.
    
    Arguments:
        k_p (float): Proportional coefficient.
        k_i (float): Integral coefficient.
        k_d (float): Derivative coefficient.
    
    Returns:
        A (numpy.ndarray): Shape (2, 2).
        B (numpy.ndarray): Shape (2,).
    """
    ...

In [ ]:
grader.check("q2a-closedloop-code")

In [ ]:
w = lambda t, x: 4.3e-6

t = np.linspace(0, 40*60, 2000)  # In seconds
theta_init = np.linspace(-0.5*np.pi, 0.5*np.pi, 5)
theta_dot_init = np.linspace(-1e-3, 1e-3, 5)
init_states = np.stack(np.meshgrid(theta_init, theta_dot_init)).reshape(2, -1).T

k = np.array([
    [0, 5, 0],
    [2000, 5, 0],
    [2000, 5, 4e5],
])

for k_p, k_i, k_d in k:
    A, B = make_closed_loop_sys(k_p, k_i, k_d)
    plt.figure(figsize=(10, 6))
    plt.suptitle(f"System simulation with K_p={k_p}, K_i={k_i}, and K_d={k_d}")
    _, phase_ax, theta_ax = simulate_satellite(A, B, w, t, init_states, scale=0.1, step=200)
    
    phase_ax.set_title(f"Phase Portrait of PID Controlled System with $K_p={k_p}, K_i={k_i}$, and $K_d={k_d}$")
    theta_ax.set_title(f"Angular Displacement of PID Controlled System with $K_p={k_p}, K_i={k_i},$ and $K_d={k_d}$")

Let's understand what these controllers are doing in the Laplace domain.
First, fill out the cell below to find the poles of $H_{w\to\theta}(s)$ for each triplet of coefficients using [`scipy.signal.ss2zpk`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ss2zpk.html#scipy.signal.ss2zpk). Don't worry if you get a warning about `BadCoefficients` as long as the plot shows up. We then plot these poles on the complex plane.

In [ ]:
C, D = [[1, 0]], [[0]]
def find_zeros_poles(k_p, k_i, k_d):
    A, B = make_closed_loop_sys(k_p, k_i, k_d)
    B = B.reshape((-1, 1))
    zpk = ...
    zeros, poles, _ = zpk
    return zeros, poles

In [ ]:
plt.figure(figsize=(8, 8))
for k_p, k_i, k_d in k:
    zeros, poles = find_zeros_poles(k_p, k_i, k_d)
    assert len(zeros) == 0
    print('Closed-loop poles with k_p = {}, k_i = {}, k_d = {}:'.format(k_p, k_i, k_d), poles)
    label = r'$k_p = {}, k_i = {}, k_d = {}$'.format(k_p, k_i, k_d)
    plt.scatter(poles.real, poles.imag, marker='x', s=100, label=label)
plt.xlabel('$\mathfrak{Re}$')
plt.ylabel('$\mathfrak{Im}$')
plt.legend()

<!-- BEGIN QUESTION -->

For second-order systems, we can express the poles as the roots of $s^2 + 2\zeta \omega_n s + \omega_n^2 = 0$, where $\omega_n$ is the *natural frequency* and $\zeta$ is the *damping ratio*.
These quantities are useful for predicting oscillations in the system's step response:
* $\zeta = 0$ is an undamped system. The step response will oscillate with frequency $\omega_n$ forever.
* $0 < \zeta < 1$ is an underdamped system. The step response will oscillate, but with an exponentially decaying envelope.
* $\zeta = 1$ is a critically damped system and $\zeta > 1$ is an overdamped system. In both cases, the step response will not oscillate, but a critically damped system will converge to equilibrium in the minimum amount of time without overshooting.

**Questions**:
1. Find the ranges of $\zeta \ge 0$ corresponding to if the poles are imaginary, complex, equal, or real.
1. Classify each set of coefficients $(k_p, k_i, k_d)$ shown above in the figure as undamped, underdamped, critically damped, and overdamped.
1. In general, how does $k_p, k_i, k_d$ each affect the damping ratio for the closed-loop system with PID? Feel free to add new coefficient triplets $(k_p, k_i, k_d)$ to our test values array `k` or find an expression for $\zeta$ directly in terms of $k_p$, $k_i$, $k_d$, $I_y$, and $I_f$.
1. How does each pole's distance from the imaginary axis affect the error decay rate?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Finally, in the cell below, with $k_p = 2000, k_i = 5$, choose $k_d$ to place the poles so that the system is critically damped. Leave `k_d` as a combination of the variables `k_p`, `k_i`, `I_y`, and `I_f`.
Plot the trajectories, and confirm that there are no oscillations.

Remember to reshape `B` for [`scipy.signal.ss2zpk`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ss2zpk.html#scipy.signal.ss2zpk) like we did to calculate the poles last time.

In [ ]:
k_p = ...
k_i = ...
k_d = ...
A, B = make_closed_loop_sys(k_p, k_i, k_d)
zeros, poles, _ = ...
assert np.allclose(np.imag(poles), 0)
plt.figure(figsize=(10, 6))
_, phase_ax, theta_ax = simulate_satellite(A, B, w, t, init_states, scale=0.1, step=200)

phase_ax.set_title("Phase portrait of critically damped closed loop system.")
theta_ax.set_title("Angular Displacement of critically damped closed loop system.")

<!-- END QUESTION -->

### Q2(b): Noise Insensitivity

Up to now, we've assumed we have sensors which we can use to sense the angular displacement and the angular velocity perfectly.
However, in reality, all sensors have some noise.

<span>
    <img src="diagrams/closed-loop-noise.png" />
    <center>Closed-loop system with sensor noise.</center>
</span>

This system has two inputs $(w(t), n(t))$ mapping to one output $\theta(t)$, as opposed to the single-input single-output (SISO) systems we've focused on in EE 120.
Thanks to superposition, we can compute the effect of each input on $\theta$ separately by zeroing the other input.
The transfer function of $n \to \theta$ is:

$$\Theta(s) = -\frac{1}{I_y s^2} I_f s C(s) \left[\Theta(s) + N(s)\right] \implies H_{n\to\theta}(s) \triangleq \frac{\Theta(s)}{N(s)} = -\frac{I_f s C(s)}{I_y s^2 + I_f s C(s)},$$

so the overall system follows:

$$\Theta(s) = H_{w\to\theta}(s) W(s) + H_{n\to\theta}(s) N(s),$$

where $H_{w\to\theta}(s)$ is as derived in 2(a).
In the absence of a good noise model, we'll assume $n(t)$ has samples drawn from a zero-mean normal distribution.
First, we'll look at $|N(i\omega)|$, the spectrum of $n(t)$.

In [ ]:
samples, sigma = 10**5, 2
noise = sigma*np.random.randn(samples)
plt.figure(figsize=(10, 6))
plt.plot(np.linspace(-samples/2, samples/2, samples),
         np.abs(np.fft.fftshift(np.fft.fft(noise))))
plt.xlabel('DFT index')
plt.ylabel('DFT coefficient magnitude')

<!-- BEGIN QUESTION -->

**Question: Why might we call $n(t)$ _additive white Gaussian noise_ (AWGN)?**

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Because the disturbance is typically a low-frequency signal (in this case, just a constant with DC content), the noise dominates at high frequencies.
Therefore, we would like $H_{n\to\theta}(s)$ to act as a lowpass filter.

In the cell below, for each set of PID coefficients, compute the frequency response of $H_{n\to\theta}(s)$ using [`scipy.signal.bode`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.bode.html#scipy.signal.bode).
Use the numerator-denominator polynomial representation of $H_{n\to\theta}(s)$ -- the two element `system` argument. Also make sure to specify what frequencies `w` we calculate this at.

A [Bode magnitude plot](https://en.wikipedia.org/wiki/Bode_plot) is simply the magnitude response plotted on a log-log scale, where the magnitude is in units of decibels:

$$20 \log_{10} |H_{n\to\omega}(i\omega)|$$

Bode plots are often more informative when the frequencies a system can be subject to vary by orders of magnitude, as is the case with AWGN.
In addition, there are good [straight-line](https://inst.eecs.berkeley.edu/~ee16b/sp18/dis/4A/ans4A.pdf) approximations for drawing Bode plots by hand.

In [ ]:
freqs = np.logspace(-3, 2, 1000)  # Frequencies to plot
plt.figure(figsize=(10, 6))
for k_p, k_i, k_d in k:
    num = ...
    den = ...
    bode = ...
    freqs, magnitude, _ = bode
    label = r'$k_p = {}, k_i = {}, k_d = {}$'.format(k_p, k_i, k_d)
    plt.semilogx(freqs, magnitude, label=label)
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Magnitude Response [dB]')
plt.legend()

**Question: Which coefficients exhibit the best lowpass behavior?**

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Q3: Nonlinear Control

As flexible and easy to design as it is, linear control like PID may not be suitable in many practical systems:
1. The flywheel we analyzed earlier cannot rotate arbitrarily fast. Instead, its speed saturates, meaning there is a constraint on what controls we can feasibly apply.
2. Some controls, like satellite thrusters, only have discrete states like "on" or "off", since it may not be cost-effective to run the engine at a fraction of its maximum power.

These cases introduce nonlinearities into the overall system, so all of our LTI system analysis is no longer valid.
For example, two LTI systems in series can be interchanged with no effect on the overall system, but this is not true with a nonlinear system.
Nevertheless, we can try simulating the nonlinear system shown in the figure below.

<center>
    <img src="diagrams/nonlinear.png" style="width: 80%" />
</center>

$$f(e) = \begin{cases}
\tau \mathrm{sgn}\{e\} & |e| \geq \delta \\
0 & |e| < \delta
\end{cases}$$
is a **relay** representing a thruster applying a constant torque $\tau > 0$ in the direction to reduce the error $e$.
The **deadband** is the region $|e| < \delta$ where no control is applied.
Without deadband, the controller would "chatter" as $e$ and $f$ rapidly change signs, wearing out the controller.

<center>
    <img src="diagrams/relay-with-deadband.png" style="width: 40%" />
    <center>Relay with deadband.</center>

</center>

In the cell below, fill in the function `f` which represets the relay with a deadband.

The generated plots will show the phase portrait of the system as well as the angular displacement over time. The boundaries of the relay's deadband are plotted with orange dashed lines.

In [ ]:
t = np.linspace(0, 2*60*60, 2000)  # In seconds
theta_init = np.linspace(-0.5*np.pi, 0.5*np.pi, 5)
init_states = np.stack(np.meshgrid(theta_init, [0])).reshape(2, -1).T

def f(e, tau=10, delta=0.05*np.pi):
    ...

u = lambda t, x: w(t, x) + f(-x[0])

plt.figure(figsize=(10, 6))
_, phase_ax, theta_ax = simulate_satellite(A_sat, B_sat, u, t, init_states, scale=3, minlength=1.5)
theta_ax.plot([t[0]/3600, t[-1]/3600], [0.05*np.pi]*2, linestyle='dashed',
              color='tab:orange', label='Deadband')
theta_ax.plot([t[0]/3600, t[-1]/3600], [-0.05*np.pi]*2, linestyle='dashed',
              color='tab:orange')
theta_ax.legend()

<!-- BEGIN QUESTION -->

**Question: Speculate on the stability of the nonlinear system. In particular, what do you notice about the trajectory at steady state relative to the deadband?**

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Conclusion

In this lab, we've seen that feedback is a very powerful tool for stabilizing systems.
In particular, we looked at how linear control can be used to track a reference signal robustly, even with environmental disturbances and sensor noise.
We also looked at how a linear control law like PID can be tuned to place the closed-loop poles and achieve a desired response in the time-domain (*e.g.*, an overdamped response).
Finally, we took a brief look at nonlinear control, and how it qualitatively differs from linear control.

## References

* [1] Gerlach, O.H. "Altitude Stabilization and Control of Earth Satellites", March 1965.
* [2] [EE 221A Lecture Notes](http://inst.eecs.berkeley.edu/~ee221a/fa19/)
* [3] [EE 222 Lecture Notes](http://inst.eecs.berkeley.edu/~ee222/sp20/)
* [4] [EE 120 Notes](https://inst.eecs.berkeley.edu/~ee120/fa19/)
* [5] [Feedback Systems](http://www.cds.caltech.edu/~murray/amwiki/index.php/Second_Edition)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Please upload the zip file produced by the result of this command to Gradescope.

In [ ]:
grader.export(force_save=True, run_tests=True)